# Importing Modules

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import mysql.connector as mycon
from prompt_toolkit.styles import Attrs
from tabulate import tabulate
from qrcodereader import scan_qr_from_webcam

# Calling MySQL

In [3]:
cob = mycon.connect(
    host='localhost', 
    user='root',
    password='<password>',
    database = 'library',
    auth_plugin='mysql_native_password'
)

# Creating Read Function
This function will display all the books available in the library

In [ ]:
def read(filter):
    tab = cob.cursor()
    match filter:
        case 1:
            tab.execute("SELECT * FROM books")
        case 2:
            tab.execute("SELECT * FROM books WHERE status = 'A'")
        case 3:
            tab.execute("SELECT * FROM books WHERE status = 'B'")
    
    bookLis = []
    for i in tab:
        bookLis.append(i)
    print(tabulate(bookLis, headers=["Book ID", "Name of Book", "Shelf", "Status"]))
read(3)

# Creating Specific Read Function
This function will display specific books available in the library


**Values Taken From User:**
1. Book ID

In [ ]:
def specificRead(bookID):
    tab = cob.cursor()
    tab.execute(f'SELECT * FROM books WHERE bookID = {bookID}')
    for i in tab:
        if i[3] == "A":
            print(f"Book Name: {i[1]}\nShelf: {i[2]}\nStatus: Available")
        elif i[3] == "B":
            print(f"Book Name: {i[1]}\nShelf: {i[2]}\nStatus: Unavailable")
specificRead(1001)

# Creating a Create Function
This function will insert books in the library using QR Code

In [ ]:
def insertBook():
    qrval = scan_qr_from_webcam()
    try:
        qrval = qrval.split(",")
        bookID = qrval[0].strip()
        bookName = qrval[1].strip()
        shelf = qrval[2].strip()
        status = qrval[3].strip()
    
    except:
        print("Could Not Scan QR Code Properly, Try Insering Manually...")
        bookId = int(input("Enter Book ID: "))
        bookName = input("Enter Book Name: ")
        shelf = input("Enter Shelf: ")
        status = input("Enter Status: ")
    
    finally:
        tab = cob.cursor()
        tab.execute(f'INSERT INTO books VALUES ({bookID}, "{bookName}", "{shelf}", "{status}")')
        cob.commit()
        print("Successfully Added to Library")
insertBook()